In [ ]:
import polars as pl
import plotly.express as px

df=pl.read_csv("releves/debit.csv")
df = df.with_columns(
   pl.col("releve").str.to_datetime("%Y-%m-%d")
)
df = df.group_by("releve").agg(pl.col("debit").sum())
df = df.select(pl.col(["releve","debit"]))

df_debit=df
print(df)




In [ ]:
df=pl.read_csv("releves/credit.csv")
df = df.with_columns(
   pl.col("releve").str.to_datetime("%Y-%m-%d")
)
df = df.with_columns((pl.when(pl.col("poste")=="salaire").then(pl.col("credit")).otherwise(0)).alias("salaire"))
df = df.with_columns((pl.when(pl.col("poste")=="savings").then(pl.col("credit")).otherwise(0)).alias("savings"))
df = df.with_columns((pl.col("salaire")+pl.col("savings")).alias("salaire+savings"))
df = df.group_by("releve").agg(pl.col("salaire").sum(),pl.col("savings").sum(),pl.col("salaire+savings").sum())
df = df.select(pl.col(["releve","salaire","savings","salaire+savings"]))
print(df)
df_credit=df


In [ ]:
df = df_debit.join(df_credit,on="releve")
df = df.sort(pl.col("releve"),descending=False)
df_join=df
with pl.Config(tbl_rows=2000):
       print(df)

# df = 

# df = df.select(list(map(pl.col,["date","releve","salaire","savings","debits","nature","poste"])))

# df = df.group_by(pl.col("releve")).agg(
#     pl.col("salaire").sum(),
#                                       pl.col("savings").sum(),
#                                       pl.col("debits").sum())


# df=df.sort(pl.col("releve"))


In [ ]:
df=df_join
fig=px.line(df,x="releve",
            y=[
                "savings",
                "salaire",
                # "salaire+savings",
                "debit"
            ],
            markers=True
              )
fig.show()


In [ ]:
df=df_join
df = df.with_columns( (pl.col("salaire+savings")-pl.col("debit")).alias("delta") )
print(df)
df.sum()